In [1]:
# Import all the essentials

import pandas as pd
import json
import os
import matplotlib.pyplot as plot
import numpy as np   # for linear algebra
import pandas as pd  # for data frame manipulations

In [2]:
# This is for the k prototype

!pip install kmodes

from kmodes.kprototypes import KPrototypes 
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

In [3]:
# now to open the files in a list

arr = os.listdir("./Partial_Files")

all_data = []

for file in arr:
    with open ("./Partial_Files/" + str(file)) as data_file:
        data = json.load(data_file)
        all_data.append(data)

In [4]:
# we only want certain columns to compare, so we only normalize the columns we want

all_df = []

for data in all_data:
    df = pd.json_normalize(data)[["Product.ProductName", "ReferenceData.IllustrationID", "PolicyScalars.PremiumAmountAnnualized", "Insured1.AgeAtIssue",
                                     "PolicyScalars.FaceAmountTotal", 'Insured1.Sex']]
    all_df.append(df)

In [5]:
# These are additional columns that I want to add, but somehow can't...
# Some help adding these would be greatly appreciated

#                               [["Product.ProductName", "ReferenceData.IllustrationID", "PolicyScalars.PremiumAmountAnnualized", 
#                                 "Insured1.AgeAtIssue", "PolicyScalars.FaceAmountTotal", "PolicyScalars.FaceAmountBase", 
#                                 'Insured1.Sex', "PolicyScalars.PremiumMode", "PolicyScalars.PremiumAmountModal",
#                                 "PolicyScalars.PremiumAmountModal", "PolicyScalars.PremiumAmountModal", 
#                                 "PolicyScalars.PremiumDuration", "PolicyScalars.PremiumTarget", 
#                                 "PolicyScalars.PremiumMinimum", "PolicyScalars.HasLoans", "PolicyScalars.HasWithdrawals", 
#                                 "PolicyScalars.HasDistributions", "PolicyScalars.HasCreditingRates",
#                                 "PolicyScalars.HasCreditingRatesGross", "PolicyScalars.HasCreditingRatesNet", 
#                                 "PolicyScalars.IsQualifiedPlan" 
#                               ]]

In [6]:
# Read in the additional data excel files: 
additional_df = pd.read_excel("Additional.xlsx")

In [7]:
# Concatenate the files into a data frame and merge in the result,
# then we drop the illustration ID column because it's simply not useful for grouping people

result = pd.concat(all_df)
result = result.merge(additional_df, on='ReferenceData.IllustrationID')
result.drop(['ReferenceData.IllustrationID'],axis='columns', inplace=True )

In [8]:
# describe is a neat command to find the numerical data in a dataframe.

result.describe()

,PolicyScalars.PremiumAmountAnnualized,Insured1.AgeAtIssue,PolicyScalars.FaceAmountTotal
count,208.000000,208.000000,2.080000e+02
mean,28415.172548,47.716346,9.848302e+05
std,62458.113036,12.498553,2.831068e+06
min,0.000000,20.000000,5.000000e+04
25%,3220.000000,37.750000,2.500000e+05
50%,9300.000000,47.000000,5.000000e+05
75%,25000.000000,56.000000,1.000000e+06
max,500000.000000,77.000000,3.800000e+07


In [9]:
# there's a whole essay on transforming NANs into values using a KNN method,
# I will look into it, but in the mean time, I will just change NANs into
# "NANs" strings, so they can be executed as normal data.

result['LISTBILL'] = result['LISTBILL'].astype(str)
result['OFFICE'] = result['OFFICE'].astype(str)
result['PARENT_OFFICE_NAME'] = result['PARENT_OFFICE_NAME'].astype(str)
result['DISTRIBUTION_CHANNEL_NAME'] = result['DISTRIBUTION_CHANNEL_NAME'].astype(str)

In [10]:
# Making sure there is no null values anymore.

result.isnull().sum()

Product.ProductName                      0
PolicyScalars.PremiumAmountAnnualized    0
Insured1.AgeAtIssue                      0
PolicyScalars.FaceAmountTotal            0
Insured1.Sex                             0
Policy_Number                            0
LISTBILL                                 0
OFFICE                                   0
PARENT_OFFICE_NAME                       0
DISTRIBUTION_CHANNEL_NAME                0
dtype: int64

In [11]:
# Now to transform the dataframe into a numpy 
# Because K-Prototype only works with numpy values

data_array = result.values
data_array[:, 1] = data_array[:, 1].astype(float)
data_array[:, 2] = data_array[:, 2].astype(float)
data_array[:, 3] = data_array[:, 3].astype(float)

In [12]:
# This is just the syntax for k prototyping
# There's another essay on finding the best number of clusters, 
# I will again look into it, but for now just take 3

kproto = KPrototypes(n_clusters=3, verbose=2,max_iter=20)
clusters = kproto.fit_predict(data_array, categorical=[0, 4, 5, 6, 7, 8, 9])

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 62, ncost: 656924787255749.5
Run: 1, iteration: 2/20, moves: 35, ncost: 358761398720731.7
Run: 1, iteration: 3/20, moves: 23, ncost: 131829567326954.94
Run: 1, iteration: 4/20, moves: 12, ncost: 111855731668086.28
Run: 1, iteration: 5/20, moves: 6, ncost: 87255422090975.1
Run: 1, iteration: 6/20, moves: 0, ncost: 87255422090975.1
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/20, moves: 23, ncost: 646450643753717.5
Run: 2, iteration: 2/20, moves: 20, ncost: 351538983248536.94
Run: 2, iteration: 3/20, moves: 16, ncost: 131829567326954.94
Run: 2, iteration: 4/20, moves: 12, ncost: 111855731668086.28
Run: 2, iteration: 5/20, moves: 6, ncost: 87255422090975.1
Run: 2, iteration: 6/20, moves: 0, ncost: 87255422090975.1
Init: initializing centroids
Init: 

In [13]:
# Cluster centroids are like where the most people have in common with each other,
# printing it out just so we can look at its values (this is where the NANs become a problem)

print(kproto.cluster_centroids_)

[['199500.00000000006' '44.0' '38000000.0' 'PIA6' 'Male' 'VF53729140'
  '            ' 'PG000' 'PRODUCER GROUP RLO' 'REG LIFE OFFICES']
 ['19730.618756218904' '47.711442786069654' '638129.815920398' 'PDXIUL2'
  'Male' 'VF53768800' '            ' 'T372J'
  'Transamerica Financial Advisors Inc' 'REG LIFE OFFICES']
 ['290833.58666666667' '48.5' '6430096.833333333' 'PACHORIZONIUL' 'Male'
  'VF53795440' '            ' 'nan' 'nan' 'WIREHOUSE']]


In [14]:
# We need to attach the clusters to the original data frame
# to see if we can find any patterns

cluster_dict = []
for c in clusters:
    cluster_dict.append(c)

In [15]:
# Make another column for the clusters:

result['cluster']=cluster_dict

In [16]:
result[result['cluster']== 0].head(10)

,Product.ProductName,PolicyScalars.PremiumAmountAnnualized,Insured1.AgeAtIssue,PolicyScalars.FaceAmountTotal,Insured1.Sex,Policy_Number,LISTBILL,OFFICE,PARENT_OFFICE_NAME,DISTRIBUTION_CHANNEL_NAME,cluster
101,PIA6,199500.0,44,38000000.0,Male,VF53729140,,PG000,PRODUCER GROUP RLO,REG LIFE OFFICES,0


In [17]:
result[result['cluster']== 1].head(10)

,Product.ProductName,PolicyScalars.PremiumAmountAnnualized,Insured1.AgeAtIssue,PolicyScalars.FaceAmountTotal,Insured1.Sex,Policy_Number,LISTBILL,OFFICE,PARENT_OFFICE_NAME,DISTRIBUTION_CHANNEL_NAME,cluster
0,VFVENTURE,8359.00,61,200000.0,Male,VF80619980,1A01R42A,0MM20,FINANCIAL DESIGNS,M FINANCIAL,1
1,PDXIUL2,12000.00,35,300000.0,Female,VF53838600,,T372J,Transamerica Financial Advisors Inc,WFG,1
2,PDXIUL2,21643.43,27,400000.0,Male,VF53764210,,T372J,Transamerica Financial Advisors Inc,WFG,1
3,PDXIUL2,6552.00,51,300000.0,Male,VF53756940,,T372J,Transamerica Financial Advisors Inc,WFG,1
4,PSVUL2,11504.00,69,300000.0,Female,VP66446660,,0C220,CAPITAS-SMITH COMPANIES LTD,WIREHOUSE,1
5,PSVUL2,11504.00,69,300000.0,Female,VP66446660,,P725A,MORGAN STANLEY (MSSB),WIREHOUSE,1
6,PSVUL2,7800.00,44,1000000.0,Female,VP66452840,,PG000,PRODUCER GROUP RLO,REG LIFE OFFICES,1
7,PDXIUL2,9600.00,36,300000.0,Male,VF53745330,,T372L,Transamerica Financial Advisors Inc,WFG,1
8,PDXIUL2,1800.00,24,300000.0,Male,VF53801540,,T372J,Transamerica Financial Advisors Inc,WFG,1
9,PACADMIRALVUL,0.00,52,750000.0,Male,VP66442930,,P931A,WELLS FARGO CLEARING SERVICES,WIREHOUSE,1


In [18]:
result[result['cluster']== 2].head(10)

,Product.ProductName,PolicyScalars.PremiumAmountAnnualized,Insured1.AgeAtIssue,PolicyScalars.FaceAmountTotal,Insured1.Sex,Policy_Number,LISTBILL,OFFICE,PARENT_OFFICE_NAME,DISTRIBUTION_CHANNEL_NAME,cluster
85,PIA6,65001.56,56,10000000.0,Male,VF53803720,nan,nan,nan,nan,2
109,PACHORIZONIUL,500000.00,42,7279655.0,Male,VF53795440,,0CB20,"Lion Street Private Client, Llc",WIREHOUSE,2
110,PACHORIZONIUL,500000.00,42,7279655.0,Male,VF53795440,,P931B,WELLS FARGO CLEARING SERVICES,WIREHOUSE,2
163,PDXIUL2,330000.00,65,4021271.0,Male,VF53763650,nan,nan,nan,nan,2
174,PDXIUL2,250000.00,46,5000000.0,Male,VF53854870,,T372J,Transamerica Financial Advisors Inc,WFG,2
194,PSVUL2,99999.96,40,5000000.0,Female,VP66458690,,IFP00,IND FINCL PROFESSIONAL RLO,REG LIFE OFFICES,2


In [19]:
# So obviously this is incomplete, cluster #1 has literally just 1 guy.

# Something we can do to improve upon that is :
# 1. actually work with the nans, instead of just bypassing them
# 2. get a bigger sample
# 3. find a more correct cluster size.